# DO NOT RUN UNTIL FINALISED  
## CNN SKELETON 
some variable names are to be changed depending on original dataset

In [ ]:
#IMPORTING LIBRARIES
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from keras import layers

DEFINING TRAIN AND TEST DATASETS, AND VISUALISING SOME IMAGES WITH THEIR LABEL

In [ ]:
# To make this notebook’s output stable across runs
np.random.seed(50)
tf.random.set_seed(50)

# Load the Dataset  -  I am calling it full in case we want to cut down the train images to 6250 for all train categories
(X_train_full, y_train_full)  = # insert train datasets X_train is the list of images, y_train is the list of labels
(X_test, y_test) = # insert same thing but for test datasets

# Normalize pixel values to be between 0 and 1
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255

""" THIS MAY BE USEFUL DEPENDING ON HOW WE LOAD THE IMAGES, WETHER AS ONLY ONE BIG TABLE OR ALREADY DIVIDED INTO TEST AND TRAIN
# Split the full training set into a training set and a (smaller) test set
# The first 6250 images and labels will be used as the training set
X_train, y_train = X_train_full[6250:], y_train_full[6250:]"""



#THE FOLLOWING CODE NEEDS TO BE HEAVILY REWRITTEN DEPENDING ON HOW WE STRUCTURE OUR TABLE CONATINING THE IMAGES!!!!!!!
#THE FOLLOWING CODE NEEDS TO BE HEAVILY REWRITTEN DEPENDING ON HOW WE STRUCTURE OUR TABLE CONATINING THE IMAGES!!!!!!!
#THE FOLLOWING CODE NEEDS TO BE HEAVILY REWRITTEN DEPENDING ON HOW WE STRUCTURE OUR TABLE CONATINING THE IMAGES!!!!!!!
category = ['Benign', 'Malignant', 'Undetected']

# visualising images with numeric tags and names  
def display_images_with_labels(images, labels, category, num_images=10, tagnum = None):
    plt.figure(figsize=(10,10))
    # select only images with specified tag number
    if tagnum >=0 and tagnum<3: 
        indices = np.where(np.array([str(label) for label in labels]) == str(tagnum))[0][:num_images]
    # otherwise select randomly
    else: 
        indices = np.random.choice(range(len(images)), num_images, replace=False)
    for i, index in enumerate(indices):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[index], cmap=plt.cm.binary)
         # if there is a list of class names, show both tag number and class name
        if class_names:
            label_with_name = f"{labels[index]} ({class_names[labels[index]]})"
            plt.xlabel(label_with_name)
        # othwerwise visualise only tag number
        else:
            plt.xlabel(labels[index])
    plt.show()

display_images_with_labels(X_train, y_train, class_names, num_images=25, category = 0)



CREATING THE MODEL AND INITIALISING DATASETS
## MODEL NEEDS TO BE MODIFIED DEPENDING ON THIS SPECIFIC CASE
This code is just an example of layers, the final composition of the model is to be decided.

In [ ]:
## Creating the models
# Lower Model
lower_model = keras.Sequential()
lower_model.add(keras.Input(shape=(28, 28, 1)))  # 28x28 B&W

lower_model.add(layers.Conv2D(32, 3, activation="relu", padding='same', kernel_initializer='he_normal'))
lower_model.add(layers.MaxPooling2D(2))

lower_model.add(layers.Conv2D(64, 3, activation="relu", padding='same', kernel_initializer='he_normal'))
lower_model.add(layers.MaxPooling2D(2))

# Upper Model
upper_model = keras.Sequential()
upper_model.add(layers.Flatten()) 

upper_model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
upper_model.add(layers.Dense(10, activation='softmax'))

# Prepare the training dataset.
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# Optimizer for the lower layers
lower_optimizer = keras.optimizers.SGD(learning_rate=1e-4)
# Optimizer for the upper layers
upper_optimizer = keras.optimizers.Nadam(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy()

# Prepare the testing dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
val_dataset = val_dataset.batch(batch_size)



INITIALISING METRICS AND RUNNING THE MODEL

In [ ]:
# Initialize metrics at the start of each epoch
loss_avg = keras.metrics.Mean()
accuracy = keras.metrics.SparseCategoricalAccuracy()
val_loss_avg = keras.metrics.Mean()
val_accuracy = keras.metrics.SparseCategoricalAccuracy()

# Training Loop
epochs = 5
for epoch in range(epochs):
    print(f"\nStart of epoch {epoch}")

    # Iterate over the batches of the training dataset
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        # Calculating the intermediate activations of the lower and upper model 
        with tf.GradientTape(persistent=True) as tape:
            lower_logits = lower_model(x_batch_train, training=True)
            logits = upper_model(lower_logits, training=True)
        # Calcutating the loss
            loss_value = loss_fn(y_batch_train, logits)

        # Gradient computation for both the models
        lower_grads = tape.gradient(loss_value, lower_model.trainable_weights)
        upper_grads = tape.gradient(loss_value, upper_model.trainable_weights)
        
        # Apply optimizers
        lower_optimizer.apply_gradients(zip(lower_grads, lower_model.trainable_weights))
        upper_optimizer.apply_gradients(zip(upper_grads, upper_model.trainable_weights))

        # Update training metrics
        loss_avg.update_state(loss_value)
        accuracy.update_state(y_batch_train, logits)

    # Print the mean training loss and accuracy over the epoch
    train_loss = loss_avg.result()
    train_accuracy = accuracy.result()
    print(f"Training loss over epoch: {float(train_loss):.4f}")
    print(f"Training accuracy over epoch: {float(train_accuracy):.4f}")


    # Perform validation at the end of the epoch
    for x_batch_val, y_batch_val in val_dataset:
        lower_val_logits = lower_model(x_batch_val, training=False)
        val_logits = upper_model(lower_val_logits, training=False)
        val_loss_value = loss_fn(y_batch_val, val_logits)
    
        val_loss_avg.update_state(val_loss_value)
        val_accuracy.update_state(y_batch_val, val_logits)
    
    # Compute the mean validation loss and accuracy for the epoch
    val_loss = val_loss_avg.result()
    val_acc = val_accuracy.result()
    print(f"Validation loss: {float(val_loss):.4f}")
    print(f"Validation accuracy: {float(val_acc):.4f}")